In [1]:
%matplotlib notebook
import cv2
import numpy as np
from matplotlib import pyplot as plt
import shutil
from pathlib2 import Path
import pandas as pd
import sys
import os
import time
#Snippet to find python modules/packages in other dirs from a jupyter notebook
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import underwater_distortions.underwater_distortions as UWdistortion

### Code snippets to help (directory structure, dataframes)

In [8]:
#create a directory if does not exist
def ensure_dir(str_file_path):
    directory = os.path.dirname(str_file_path)
    try: 
        os.makedirs(directory)
    except OSError:
        if not os.path.isdir(directory):
            raise
    return True

def create_new_db_tree(root_dir,scenarios_list,class_name,no_versions,version_offset=None):
    version_offset = 0 if version_offset==None else version_offset
    #create a dir for each scenario
    for sdir in scenarios_list:
        #create a subdir for each version
        for x in range(no_versions):
            formated_dir_num = '{:02d}'.format(x+version_offset)
            str_new_dir = (root_dir + sdir + '/true_positives/' + class_name + '/dir_' + formated_dir_num+'/')
            ensure_dir(str_new_dir)
            str_new_dir = (root_dir + sdir + '/true_negatives/' + class_name + '/dir_' + formated_dir_num+'/')
            ensure_dir(str_new_dir)
            
    return True

#create and entry with 
def create_sampleMetaData(df_source,stereo_left,stereo_right,distortion_type,param1,param2):
    new_sample = [{'scenario':df_source['scenario'],'stereo left':stereo_left,'stereo right':stereo_right,
                   'label name':df_source['label name'],'label id':df_source['label id'],
                   'roi left':df_source['roi left'],'roi right':df_source['roi right'],'synthetic':1,
                   'distortion':distortion_type,'param 1':param1,'param 2':param2}]
    return new_sample

### Global variables

In [13]:
#Original db variables
str_data_path = '../../../../data/' 
str_db_path = str_data_path+'CADDY_gestures_complete_v2_release/'
str_db_csv = str_db_path + 'CADDY_gestures_all_true_positives_release_v2.csv'

#New db variables
distortion_name = 'blur'
distortion_values = [1.5,3.0]
version_start_num = 0
str_class_type = 'true_positives'
str_new_db_path = str_data_path+'CADDY_gestures_complete_v2_release_'+distortion_name+'/'
str_new_db_csv = str_new_db_path + 'CADDY_gestures_all_true_positives_release_v2-blur.csv'

#Auxiliary variables to create new data frame
df_columns = ['scenario','stereo left','stereo right','label name','label id','roi left','roi right',
              'synthetic','distortion','param 1','param 2']
dummy_row = [{'scenario':'dummy','stereo left':'dummy','stereo right':'dummy','label name':'dummy',
              'label id':-1,'roi left':-1,'roi right':-1,'synthetic':-1,'distortion':'dummy',
              'param 1':-1,'param 2':-1}]

#### Read original database

In [4]:
#Read original file with non-altered stereo images
tmp_df = pd.read_csv(str_db_csv)
database_source_df = tmp_df.loc[tmp_df['synthetic'] == 0]
#database_source_df.head(10)

#### Create same directory structure - Execute only once per distortion

In [35]:
#Create new dir for db
ensure_dir(str_new_db_path)
#Create same subdirs as orginal database structure
scenarios_list = database_source_df.scenario.unique()
create_new_db_tree(str_new_db_path,scenarios_list,distortion_name,len(distortion_values))

True

In [16]:
#Create new csv db file
new_database_df = pd.DataFrame(dummy_row)
new_database_df  = new_database_df.loc[:,df_columns]
new_database_df.head()

,scenario,stereo left,stereo right,label name,label id,roi left,roi right,synthetic,distortion,param 1,param 2
0,dummy,dummy,dummy,dummy,-1,-1,-1,-1,dummy,-1,-1


In [17]:
#iterate through orginal file
file_cc = 0 #counter to see progress and allow for quick testing
for index, row in database_source_df.iterrows():
    if file_cc > -1:
        
        #apply distortions to each image and save the  metadata in the dataframe
        img_orig_left = cv2.imread(str_db_path[:-1] + row['stereo left'], cv2.IMREAD_COLOR)
        img_orig_right = cv2.imread(str_db_path[:-1] + row['stereo right'], cv2.IMREAD_COLOR)
        
        #debugging
        #print(row['stereo left'])
        
        #apply distortion
        for x in range(len(distortion_values)):
            
            #create distorted images and save them in corresponding dirs
            formated_dir_num = '{:02d}'.format(version_start_num+x)
            distorted_img = UWdistortion.gaussian_blur(img_orig_left,distortion_values[x])
            str_new_dir = (str_new_db_path + row['scenario'] + '/' + str_class_type + '/' + 
                          distortion_name + '/dir_' + formated_dir_num + '/')
            str_imgL_path = str_new_dir + Path(row['stereo left']).name
            cv2.imwrite(str_imgL_path,distorted_img,[cv2.IMWRITE_JPEG_QUALITY,100])            
            
            distorted_img = UWdistortion.gaussian_blur(img_orig_right,distortion_values[x])
            str_imgR_path = str_new_dir + Path(row['stereo right']).name
            cv2.imwrite(str_imgR_path,distorted_img,[cv2.IMWRITE_JPEG_QUALITY,100])
            
            #save metadata in new dataframe
            #TODO: these distortions have only one parameter
            #make the method more robust in case there is other params
            
            #Maek paths to image relative to db directory
            str_relative_imgL_path = '/'+ str_imgL_path.replace(str_new_db_path,'')
            str_relative_imgR_path = '/'+ str_imgR_path.replace(str_new_db_path,'')
            new_database_df = new_database_df.append(create_sampleMetaData(row,
                                                                           str_relative_imgL_path,
                                                                           str_relative_imgR_path,
                                                                           distortion_name,
                                                                           distortion_values[x],
                                                                           float('nan')),ignore_index=True)
            
        
        
        file_cc += 1
        sys.stdout.write("\r Processed: " + str(file_cc))
        
#erase dummy row of the new dataframe and save the df
new_database_df = new_database_df.drop(new_database_df.index[0])
new_database_df.to_csv(str_new_db_csv)
print(' AUGMENTED DATA BASE CREATED ')

 Processed: 3 AUGMENTED DATA BASE CREATED 


In [18]:
new_database_df.head()

,scenario,stereo left,stereo right,label name,label id,roi left,roi right,synthetic,distortion,param 1,param 2
1,biograd-A,/biograd-A/true_positives/blur/dir_00/biograd-...,/biograd-A/true_positives/blur/dir_00/biograd-...,num_delimiter,10,"[237,236,54,65]","[155,236,54,65]",1,blur,1.5,NaN
2,biograd-A,/biograd-A/true_positives/blur/dir_01/biograd-...,/biograd-A/true_positives/blur/dir_01/biograd-...,num_delimiter,10,"[237,236,54,65]","[155,236,54,65]",1,blur,3.0,NaN
3,biograd-A,/biograd-A/true_positives/blur/dir_00/biograd-...,/biograd-A/true_positives/blur/dir_00/biograd-...,num_delimiter,10,"[230,230,61,79]","[147,230,61,79]",1,blur,1.5,NaN
4,biograd-A,/biograd-A/true_positives/blur/dir_01/biograd-...,/biograd-A/true_positives/blur/dir_01/biograd-...,num_delimiter,10,"[230,230,61,79]","[147,230,61,79]",1,blur,3.0,NaN
5,biograd-A,/biograd-A/true_positives/blur/dir_00/biograd-...,/biograd-A/true_positives/blur/dir_00/biograd-...,num_delimiter,10,"[222,230,72,75]","[139,230,72,75]",1,blur,1.5,NaN
